# <strong>Non-customer recovery in e-commerce</strong>

### How does the work of the data analyst/business intelligence help the commercial team to improve the platform conversion?<br>
When selling products and services in an e-commerce, it is normal and expected to lose sales during the purchase process. This can be abstracted into a funnel, called a sales funnel, and contain partitions, normally segmented by customer interest. Here we will use the following actions in the purchase process to define the partitions:

1) Customer enters the e-commerce;<br>
2) Customer logs in;<br>
3) Customer places a product in the cart;<br>
4) Customer choose shipping and payment method;<br>
5) Customer makes the payment.<br>

![title](images/sales_funnel2.png)
<br>

### What is the role of the data analyst/business intelligence? 
We deliver *leads* separated by the funnel stages to the commercial team for assertive action. Seeking to understand the cause of the<strong> ABANDONMENT </strong> of the sales stages and we help the customer in the process.

### Probable causes of abandonment of steps
For the first step, there is only identification by IP, which is normally used by Google Ads for remarketing. For the second stage, we can ask if the customer needs help, and what they thought of the e-commerce, as registering on the platform is an indication of interest in buying or looking for a solution to their problem. In the third step, we can look for the customer's objection not to move forward. In the fourth step, from my own experience, I say that payment difficulties or inadequate shipping options are common. In the fifth stage, we can see if there was, in fact, the forgetting of the payment or just financial problems.

## What does the professional do for the commercial?
It will search the database for this information and send the *lead* lists to the sales team. To get the information, we will go into the technical part, using Python and SQL. In this example, we will create the database for the queries that will generate the lists.

## Database creation (DB) and information query
About the Database, we have the following files:<br>
1) Table of shopping carts;<br>
2) Table of customer records;<br>
3) Table of orders;<br>
4) Table of product price;<br>
5) Table of Order Status Descriptions.<br>
All in CSV files created for this example and with the DB schema resembling a virtual store.

We will also assemble a drawing of the *schema* or database schema to understand its structure. There are three fact tables: cart, orders, and customers. The other two tables are dimension tables with data information such as the product price table and status description. They are linked by database keys forming the *schema*. The lines represent the links between the tables.

![title](images/schema.png)

# Code and explanation

In [1]:
import pandas as pd

# Importing csv tables into Pandas Dataframe
carrinho = pd.read_csv('./dados/carrinho.csv',sep=';',decimal=",")
pedidos = pd.read_csv('./dados/pedidos.csv',sep=';',decimal=",")
cliente = pd.read_csv('./dados/cliente.csv',sep=';',decimal=",")
descricao = pd.read_csv('./dados/status_description.csv',sep=';',decimal=",")
precos = pd.read_csv('./dados/product_price.csv',sep=';',decimal=",")

In [2]:
# First we look at the quality of the data.
# We will see if there are any missing values in the table that could break the lookups or change the counts.
print('Cart:\n',carrinho.isna().sum())
print('\Orders:\n',pedidos.isna().sum())
print('\Customers:\n',cliente.isna().sum())

Cart:
 cart_id        0
customer_id    0
product_id     0
quantity       0
date_added     0
dtype: int64
\Orders:
 order_id            0
customer_id        27
order_status_id     0
valor               0
date_added          0
dtype: int64
\Customers:
 customer_id    0
date_added     0
dtype: int64


In [3]:
print('\Description\n',descricao.isna().sum())
print('\Prices\n',precos.isna().sum())

\Description
 order_status_id    0
description        0
dtype: int64
\Prices
 product_id    0
price         0
dtype: int64


In [4]:
# We found problems only in the orders table, and now let's take a look at the values.
pedidos[pedidos.isna().any(axis=1)]

,order_id,customer_id,order_status_id,valor,date_added
9,2223,NaN,2,3797.42,01/10/2021 10:29
18,2232,NaN,1,445.00,01/10/2021 14:26
31,2245,NaN,1,3381.10,02/10/2021 12:12
33,2247,NaN,1,250.97,02/10/2021 15:59
69,2283,NaN,4,226.10,05/10/2021 14:38
73,2287,NaN,3,2612.39,05/10/2021 16:24
85,2299,NaN,3,711.61,06/10/2021 14:35
86,2300,NaN,2,604.20,06/10/2021 14:41
88,2302,NaN,1,2720.79,06/10/2021 15:32
180,2394,NaN,3,455.95,13/10/2021 13:20


In [5]:
# Note that there is a serious error in the order table, an order cannot be anonymous.
# But it doesn't necessarily mean that there was no payment, so we can't exclude.
# To handle these events let's put it as a special client, let's give it the number 999.
# Let's create a special client for this case with the number 999
pedidos.fillna(int(999),inplace=True)

In [6]:
# Now inserting this customer into the customer table for database integrity.
import datetime
agora = datetime.datetime.now()
agora = agora.strftime("%d/%m/%Y %H:%M")
cliente_especial = pd.DataFrame(data={'customer_id':[999],'date_added':[agora]})
cliente_especial.head()
cliente = pd.concat([cliente,cliente_especial],ignore_index=True)
cliente.tail()

,customer_id,date_added
605,606,31/10/2021 14:52
606,607,31/10/2021 14:52
607,608,31/10/2021 17:14
608,609,31/10/2021 17:14
609,999,21/07/2022 12:36


In [7]:
# Checking the exchange of missing values for client 999, if you find something has been changed.
pedidos.loc[pedidos['customer_id']== 999]


,order_id,customer_id,order_status_id,valor,date_added
9,2223,999.0,2,3797.42,01/10/2021 10:29
18,2232,999.0,1,445.00,01/10/2021 14:26
31,2245,999.0,1,3381.10,02/10/2021 12:12
33,2247,999.0,1,250.97,02/10/2021 15:59
69,2283,999.0,4,226.10,05/10/2021 14:38
73,2287,999.0,3,2612.39,05/10/2021 16:24
85,2299,999.0,3,711.61,06/10/2021 14:35
86,2300,999.0,2,604.20,06/10/2021 14:41
88,2302,999.0,1,2720.79,06/10/2021 15:32
180,2394,999.0,3,455.95,13/10/2021 13:20


In [8]:
# Checking insertion in the customer table.
cliente.loc[cliente['customer_id']== 999]

,customer_id,date_added
609,999,21/07/2022 12:36


In [9]:
# With the dataframes already treated, let's generate a SQL Database of type SQLITE.
import os
os.remove("recuperacao_vendas.db") if os.path.exists("recuperacao_vendas.db") else None
import sqlalchemy
con_sql = sqlalchemy.create_engine("sqlite:///recuperacao_vendas.db")
# Creating the database data
carrinho.to_sql(name='carrinho',con=con_sql,if_exists='replace')
pedidos.to_sql(name='pedidos',con=con_sql,if_exists='replace')
cliente.to_sql(name='cliente',con=con_sql,if_exists='replace')
descricao.to_sql(name='descricao',con=con_sql,if_exists='replace')
precos.to_sql(name='precos',con=con_sql,if_exists='replace')

In [10]:
# Connecting to the database.
import sqlite3
con = sqlite3.connect('recuperacao_vendas.db')
curs = con.cursor()

### Opportunity abandonment
In this step, the action is to the top of the sales filter. To filter only the customers who registered, and did not move forward in the journey, therefore, it is necessary to check all customers who took some action after registration. That is, the lists of customers that appear in the cart and order records are filtered (since there may be orders without a cart). For the example below, both lists are retrieved by subqueries on each table.

<strong> How much payback is there in this recovery? </strong><br>
The question for this question will always vary depending on the store's average ticket and customer conversion percentage. Let's say if we have 3% conversion on this action with an average ticket of 200 BRL, we will have 6 BRL from this list, approximately.

Query code:

In [11]:
abandono_de_oportunidade='''
SELECT cliente.customer_id as data FROM cliente
	WHERE
	cliente.customer_id NOT IN ( SELECT pedidos.customer_id FROM `pedidos` WHERE pedidos.customer_id IS NOT NULL ) 
	AND cliente.customer_id NOT IN ( SELECT carrinho.customer_id FROM carrinho ) 
	ORDER BY
	cliente.customer_id DESC'''
# Select all records and retrieve records
curs.execute(abandono_de_oportunidade)
dados = curs.fetchall()
names = list(map(lambda x: x[0], curs.description))
oportunidade = pd.DataFrame(dados,columns=names)

# Generate Opportunity Abandon List in Excel Table
oportunidade.to_excel('oportunidade.xlsx',index=False)

In [12]:
retorno_abandono = oportunidade.size*6 #6 BRL per contact
print('Possible returned value is R$', retorno_abandono)

Possible returned value is R$ 1464


### Cart abandonment
In this step, the action is for the second top-down part of the sales filter. To filter out only customers who stopped at the cart, and did not continue on the journey, it is necessary to check all customers who arrived at *checkout*. That is, the lists of customers that appear in the order records are filtered.

<strong> How much money is in this recovery? </strong><br>
Again, the conversion issue, there needs to be history to know predictability. This will be a percentage of the total amount left in the cart. Let's say there is 10% of the cart for the return forecast.

Query code:

In [13]:
abandono_de_carrinho='''
SELECT
    ca.cart_id, ca.date_added AS Dia, ca.customer_id, ca.product_id, ca.quantity, pp.price
FROM
	`carrinho` ca
	JOIN cliente cust ON ca.customer_id = cust.customer_id
	LEFT JOIN precos pp ON ca.product_id = pp.product_id 
WHERE
	ca.customer_id NOT IN ( 0 ) -- Admin
	AND ca.customer_id NOT IN (
		SELECT 	p.customer_id FROM  `pedidos` p WHERE p.customer_id IS NOT NULL ) -- Passaram da etapa de pedidos
ORDER BY
	'Id cliente' DESC, ca.date_added DESC'''
# Selects all records and retrieves records
curs.execute(abandono_de_carrinho)
dados = curs.fetchall()
names = list(map(lambda x: x[0], curs.description))
carrinho = pd.DataFrame(dados,columns=names)

# Generate cart abandonment list in Excel table
carrinho.to_excel('carrinho.xlsx',index=False)

In [14]:
abandono_de_carrinho = sum(carrinho['quantity']*carrinho['price'])
retorno_carrinho = abandono_de_carrinho*0.1
print('There are',carrinho['customer_id'].nunique(),'customers who abandoned their cart. \nWith a potential to sell ',retorno_carrinho,'BRL.')

There are 26 customers who abandoned their cart. 
With a potential to sell  1138.2 BRL.


### Abandoned payment
In this step, the action is for the last step of the sales filter. To filter only customers who had difficulty with payment, that is, did not make the payment. This customer deserves special treatment not to charge, but to check if there was a forgetfulness and/or want another payment option.
Note: The part of "Difficulty in shipping and/or installments" was left out of this case because there is no record in this database. An alternative is the implementation of a record of the abandoned *checkout* to understand if it was the freight or the installment, as well as taking the potential customer to the assistance of a salesperson.

Query code:

In [15]:
abandono_de_pagamento='''
SELECT
    p.customer_id, p.date_added AS Dia, p.valor
FROM
	pedidos as P
WHERE
	p.order_status_id = 1 and p.customer_id IS NOT NULL
ORDER BY
	p.date_added DESC'''

# Selects all records and retrieves records
curs.execute(abandono_de_pagamento)
dados = curs.fetchall()
names = list(map(lambda x: x[0], curs.description))
pagamento = pd.DataFrame(dados,columns=names)

# Generate payment abandonment list in Excel table
pagamento.to_excel('pagamento.xlsx',index=False)

In [16]:
pagamento_abandonado = round(pagamento['valor'].sum(),2)
retorno_pagamento = round(pagamento_abandonado*0.15,2)
print('There are', pagamento['customer_id'].nunique() ,'customers who have not paid a total of R$',pagamento_abandonado,'with a possible conversion of R$',retorno_pagamento)



There are 92 customers who have not paid a total of R$ 110472.78 with a possible conversion of R$ 16570.92


# Conclusion
Through SQL search and use of Python, it was possible to provide lists of customers already warmed up by marketing to the commercial team. With this, it is expected to increase the site's conversion and greater revenue. It's still possible to configure Python to send email with attached files to assignees automatically, but that's out of scope. We will also have a forecast of revenue increase with these actions added together.<br>
The lists will now be in the same folder as this python file.

In [17]:
resgate_total=retorno_abandono+retorno_carrinho+retorno_pagamento
valores_status = pedidos.groupby(['order_status_id'])['valor'].sum()
total_vendido = valores_status[0:2].sum()
print('The total expected conversion with the additional actions is R$',resgate_total)
print('The total sold in the month is R$',total_vendido,'with a possible increase in',round(resgate_total/total_vendido,3)*100,'%')


The total expected conversion with the additional actions is R$ 19173.12
The total sold in the month is R$ 211314.65 with a possible increase in 9.1 %


In [18]:
#Closing the database and cursor.
curs.close()
con.close()